In [ ]:
pip install sodapy

In [ ]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt
import sodapy
from sodapy import Socrata

# Top 500 Monthly Searches

The below uses the requests package just so I can have a preview of what I'm looking for with the other method

In [ ]:
endpoint = 'https://data.nashville.gov/resource/fuaa-r5cm.json'
response = requests.get(endpoint)

In [ ]:
response

In [ ]:
response.text

In [ ]:
response.json

Truth be told, I'm not totally sure why this looks different from the other example, but that's okay!

In [ ]:
#coming back later to say I figured out why -- I forgot the () after the .json

I was just informed that the "examples don't use the requests package" does not mean that I can't use the requests package so... back to requests! 

# QUESTION 1

### Make an API request that returns the months where "fire" was searched in 2016. Which month had the most searches?

In [ ]:
q1 = requests.get(endpoint+'?year=2016&query_text=fire&$limit=50000')
q1

In [ ]:
q1.text

In [ ]:
q1j = q1.json()
q1j

Okay, so at a glance I can see that August had the most searches, but I want to see if I can get it ordered... I believe I could turn it into a pandas DataFrame, but I want to try to get it with the API request alone.

In [ ]:
#credit to Katie and Phil who figured this out simultaneously
q1ordered = requests.get(endpoint+'?year=2016&query_text=fire&$order= query_count DESC')
q1jordered = q1ordered.json()
q1jordered

### question 1 answer: August had the most searches for "fire" in 2016 with 47 searches.

# QUESTION 2

### Make an API request that returns all the times a query was run more than 100 times in a month. How many times did this occur?

In [ ]:
#unsuccessful attempt -- query_count was a successful parameter for the ordering clause in the last question, but not this apparently
q2 = requests.get(endpoint+'?query_count>100')
q2j = q2.json()
q2j

In [ ]:
q2 = requests.get(endpoint+'?$where= query_count > 100&$limit=50000')
q2j = q2.json()
q2j

In [ ]:
q2jlength = len(q2j)
q2jlength

<old answer when I hadn't included the limit = 50000 parameter was 1,000>
I am truthfully surprised that this is such an even number, but the other people in my breakout room got this answer too, so... I am inclined to trust it.

new note: it's not 1,000 anymore

### question 2 answer: there were 1,262 times in the dataset where a query was run more than 100 times in a month.

# QUESTION 3

### Make another API request that returns all the times "codes" was searched more than 100 times in a month. How many times did this occur?

In [ ]:
q3 = requests.get(endpoint+'?query_text=codes&$where= query_count > 100&$limit=50000')
q3j = q3.json()
q3j

In [ ]:
q3jlength = len(q3j)
q3jlength

### question 3 answer: there were 56 times in the dataset where "codes" was searched more than 100 times in a month.

# QUESTION 4 -- take 1

### Make an API request that returns the entire Top 500 Monthly Searches dataset. Make a chart that shows the number of times "maps" was searched in a month across the entire time frame.

In [ ]:
#all of this ended up being incorrect and useless (other than learning purposes), so rather than delete it all, I'm going to convert it all to markdown for future reference

q4 = requests.get(endpoint)
q4j = q4.json()
q4j

#I'm not sure if this is going to be helpful but I'm going to try to make a DataFrame

q4df = pd.DataFrame(q4j)
q4df

len(q4df)

len(q4j)

q4df.info()

maps_query = ['maps']
maps = q4df[q4df.query_text.isin(maps_query)]
maps

#that cannot possibly be right

maps2 = q4df[q4df.query_text.str.contains('maps')]
maps2

len(maps2)

In [ ]:
#that feels better, but not...great...

OKAY. I just realized that there is a default limit of 1000 records, which in hindsight should have been obvious but WE ARE LEARNING, but I can override the limit with the $limit parameter. The data.nashville.gov page with the top 500 search results dataset says there are 43.7K rows in this dataset, so I'm going to try to just have limit=50k and see if it doesn't take 5 hours to load.

q4 = requests.get(endpoint+'?$limit=50000')
q4j = q4.json()
q4j

len(q4j)

#nice.

q4df = pd.DataFrame(q4j)
q4df

maps = q4df[q4df.query_text.isin(maps_query)]
maps

maps["month_year"] = maps['month_name'] + " " + maps['year']
maps

#going to dig into that warning message a bit more later, but for now, it looks like it did what I wanted, so I'm moving on

plt.figure(figsize=(15, 10))
plt.yticks()
plt.plot(maps['month_year'], maps['query_count'])
plt.show

#that's... weird. and the yticks are not in numerical order, and it's not a formatting issue...

q4df.describe()

#why isn't it giving me numeric details? wait I bet they're all stored as strings 

q4df.info()

#I was right oop... plus I don't have the month-year column I want so rewind a bit

q4df["month_year"] = q4df['month_name'] + " " + q4df['year']

q4df_dropped = q4df.dropna()

q4df_dropped.astype({'year':'int32', 'query_count':'int32'}).dtypes #ran error, need to drop NaN rows, added cell above

maps = q4df_dropped[q4df_dropped.query_text.isin(maps_query)]
maps

maps.describe()

#again?...
maps.info()

maps.astype({'year':'int32', 'query_count':'int32'}).dtypes

maps.info()

plt.figure(figsize=(15, 10))
plt.plot(maps['month_year'], maps['query_count'])
plt.show

# QUESTION 4 RESTART
##### now that I know a bunch more than I did, but it got messy, I'm starting this question over. I'll go back and fix the other questions with the limit=50k thing, too.

In [ ]:
q4 = requests.get(endpoint+'?$limit=50000')
q4j = q4.json()
q4j

In [ ]:
q4df = pd.DataFrame(q4j)
q4df

In [ ]:
q4df.info()

In [ ]:
q4df['month_year'] = q4df['month_name'] + " " + q4df['year']
q4df

In [ ]:
q4df.info()

In [ ]:
q4df[['year','query_count']] = q4df[['year','query_count']].apply(pd.to_numeric)
print(q4df.dtypes)

In [ ]:
maps_query = ['maps']
maps = q4df[q4df.query_text.isin(maps_query)]
maps

In [ ]:
maps.info()

In [ ]:
plt.figure(figsize=(25, 10))
plt.bar(maps['month_year'], maps['query_count'])
plt.title('Count of Search "Maps" By Month')
plt.ylabel('Query Count')
plt.xticks(rotation=60)
plt.show

In [ ]:
#the point of this project is not to make gorgeous gorgeous graphs so I'm going to call this adequate for now :)

# STRETCH QUESTIONS

# QUESTION 5
### Make an API request to pull back all the data from hubNashville (311) Service Requests (check to see how many rows you can return in a single request). Compare it to the Top 500 Monthly Searches data set. What do you observe? (This is open-ended, there isn't a specific answer for this one)

In [ ]:
#so I believe I have SoDaPy 2.1 installed, which should have no upper limit... let's give it a try.

In [ ]:
endpoint2 = 'https://data.nashville.gov/resource/7qhx-rexh.json'
q5 = requests.get(endpoint2+'?$limit=950000')
q5j = q5.json()
q5j

In [ ]:
#converting to a DataFrame just to make sure my understanding of how to read a JSON is correct, for learning purposes
q5df = pd.DataFrame(q5j)
q5df

In [ ]:
q5df.info()

### question 5 answer:

I already knew this dataset was significantly larger than the top 500 monthly searches set, but that is true in both dimensions of the output; there are over 930,000 rows, and 34 columns, while the other only had 5 columns. This one has more data types, as well; there are the appearance of boolean fields (although the entire DF is actually strings, but let's pretend for a bit). This dataset also has a native indexing system of sorts, since each row has a unique identifier assigned. This is geographical data, and appears to be compatible with geospatial mapping.

# QUESTION 6
### Find 2 new data sets on data.nashville.gov, make API requests to pull the data, and do an analysis that combines the data sets.

In [ ]:
#building permits issued
permits = 'https://data.nashville.gov/resource/3h5w-q8b7.json'
#tornado siren locations
sirens = 'https://data.nashville.gov/resource/736j-2drk.json'

In [ ]:
q6permits = requests.get(permits+'?$limit=50000')
q6permitsj = q6permits.json()
q6permitsj

In [ ]:
q6sirens = requests.get(sirens+'?$limit=50000')
q6sirensj = q6sirens.json()
q6sirensj

In [ ]:
permitsdf = pd.DataFrame(q6permitsj)
permitsdf

In [ ]:
sirensdf = pd.DataFrame(q6sirensj)
sirensdf

In [ ]:
permits_zip = permitsdf.zip.value_counts()
permits_zip = pd.DataFrame(permits_zip)
permits_zip.reset_index(inplace=True)
permits_zip.columns = ['Zip', 'Permit_Count']
permits_zip

In [ ]:
sirens_zip = sirensdf.zip.value_counts()
sirens_zip = pd.DataFrame(sirens_zip)
sirens_zip.reset_index(inplace=True)
sirens_zip.columns = ['Zip', 'Siren_Count']
sirens_zip

In [ ]:
combined = pd.merge(permits_zip, sirens_zip, on=['Zip'], how='outer')
combined

In [ ]:
combined = combined.fillna(0)
combined

##### I want to filter this to zipcodes with 0 tornado sirens, and could in theory move into the geospatial ecosystem and make a map, but would probably just make some sort of chart. Either way, I ran out of class time. I'll be back!